In [3]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [4]:
# load document
in_filename = 'data/eminem.txt'
doc = load_doc(in_filename)
print(doc[:200])

"Look, I was gonna go easy on you not to hurt your feelings"
"But I'm only going to get this one chance" (Six minutes—, six minutes—)
"Something's wrong, I can feel it" (Six minutes, Slim Shady, you'r


In [5]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [6]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['look', 'i', 'was', 'gonna', 'go', 'easy', 'on', 'you', 'not', 'to', 'hurt', 'your', 'feelings', 'but', 'im', 'only', 'going', 'to', 'get', 'this', 'one', 'chance', 'six', 'six', 'somethings', 'wrong', 'i', 'can', 'feel', 'it', 'six', 'minutes', 'slim', 'shady', 'youre', 'on', 'just', 'a', 'feeling', 'ive', 'got', 'like', 'somethings', 'about', 'to', 'happen', 'but', 'i', 'dont', 'know', 'what', 'if', 'that', 'means', 'what', 'i', 'think', 'it', 'means', 'were', 'in', 'trouble', 'big', 'trouble', 'and', 'if', 'he', 'is', 'as', 'bananas', 'as', 'you', 'say', 'im', 'not', 'taking', 'any', 'chances', 'you', 'are', 'just', 'what', 'the', 'doc', 'ordered', 'im', 'beginnin', 'to', 'feel', 'like', 'a', 'rap', 'god', 'rap', 'god', 'all', 'my', 'people', 'from', 'the', 'front', 'to', 'the', 'back', 'nod', 'back', 'nod', 'now', 'who', 'thinks', 'their', 'arms', 'are', 'long', 'enough', 'to', 'slap', 'box', 'slap', 'box', 'they', 'said', 'i', 'rap', 'like', 'a', 'robot', 'so', 'call', 'me', 'rap

In [7]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
 # select sequence of tokens
 seq = tokens[i-length:i]
 # convert into a line
 line = ' '.join(seq)
 # store
 sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 31944


In [8]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
 data = '\n'.join(lines)
 file = open(filename, 'w')
 file.write(data)
 file.close()


# save sequences to file
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

In [9]:

# load doc into memory
def load_doc(filename):
 # open the file as read only
 file = open(filename, 'r')
 # read all text
 text = file.read()
 # close the file
 file.close()
 return text
 
# load
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [10]:
import tensorflow as tf
import numpy as np
# integer encode sequences of words
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

seq_length = X.shape[1]

2023-03-07 12:01:23.250941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:

# define model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 50, input_length=seq_length))
model.add(tf.keras.layers.LSTM(100, return_sequences=True))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))
print(model.summary())

2023-03-07 12:01:27.640819: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            115300    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 2306)              232906    
                                                                 
Total params: 499,106
Trainable params: 499,106
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
250/250 [==============================] - 26s 94ms/step - loss: 6.4370 - accuracy: 0.0328
Epoch 2/100
250/250 [==============================] - 24s 96ms/step - loss: 5.9939 - accuracy: 0.0372
Epoch 3/100
250/250 [==============================] - 24s 97ms/step - loss: 5.7487 - accuracy: 0.0401
Epoch 4/100
250/250 [==============================] - 25s 100ms/step - loss: 5.5569 - accuracy: 0.0486
Epoch 5/100
250/250 [==============================] - 24s 98ms/step - loss: 5.3480 - accuracy: 0.0546
Epoch 6/100
250/250 [==============================] - 25s 101ms/step - loss: 5.1380 - accuracy: 0.0648
Epoch 7/100
250/250 [==============================] - 25s 101ms/step - loss: 5.0142 - accuracy: 0.0732
Epoch 8/100
250/250 [==============================] - 26s 102ms/step - loss: 4.8009 - accuracy: 0.0828
Epoch 9/100
250/250 [==============================] - 26s 106ms/step - loss: 4.6283 - accuracy: 0.0998
Epoch 10/100
250/250 [==============================] - 27s 108ms/st

In [13]:
from pickle import dump

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [14]:
# load doc into memory
def load_doc(filename):
 # open the file as read only
 file = open(filename, 'r')
 # read all text
 text = file.read()
 # close the file
 file.close()
 return text
 
# load cleaned text sequences
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

seq_length = len(lines[0].split()) - 1

In [17]:
from random import randint
from pickle import load
from keras.models import load_model
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		#yhat = model.predict_classes(encoded, verbose=0)
		predict_y=model.predict(encoded) 
		yhat=np.argmax(predict_y,axis=1)

		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

and thats the message that we deliver to little kids and expect them not to know what a womans clitoris is of course theyre gonna know what intercourse is by the time they hit fourth grade theyve got the discovery channel dont they we aint nothin but some of us cannibals

1/1 [==============================] - 0s 21ms/step
who cut partner but the beat meets really all on a broken motherfucker or wanna turn a pic and if hard empty without me so come on and dip bum on your lips and if you feel like i feel im but theyre gonna get it but im means i
